In [3]:
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv("C:/Users/USER/yolov8/yolov8-seg/runs/segment/train_con8/results.csv")
df.head(3)

,epoch,train/box_loss,train/seg_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),metrics/mAP50-95(M),val/box_loss,val/seg_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,1.65780,3.2735,3.2548,1.51930,0.82211,0.72367,0.69617,0.58483,0.81807,0.72011,0.69388,0.47341,0.69994,1.5850,2.6985,0.89123,0.000283,0.000283,0.000283
1,2,0.92100,1.4139,1.6942,0.98262,0.96448,0.77298,0.81965,0.69444,0.96448,0.77298,0.81941,0.66227,0.75255,1.1885,2.1867,0.93700,0.000583,0.000583,0.000583
2,3,0.83914,1.1888,1.4551,0.95567,0.86124,0.81495,0.80747,0.68315,0.87209,0.80070,0.79869,0.47669,0.71941,1.7702,2.2854,0.89041,0.000883,0.000883,0.000883


In [19]:
df.columns

Index(['                  epoch', '         train/box_loss',
       '         train/seg_loss', '         train/cls_loss',
       '         train/dfl_loss', '   metrics/precision(B)',
       '      metrics/recall(B)', '       metrics/mAP50(B)',
       '    metrics/mAP50-95(B)', '   metrics/precision(M)',
       '      metrics/recall(M)', '       metrics/mAP50(M)',
       '    metrics/mAP50-95(M)', '           val/box_loss',
       '           val/seg_loss', '           val/cls_loss',
       '           val/dfl_loss', '                 lr/pg0',
       '                 lr/pg1', '                 lr/pg2'],
      dtype='object')

In [18]:
df.shape

(291, 20)

In [21]:
df.columns = df.columns.str.replace(' ','') #KeyError(as special character)
df.columns = df.columns.str.replace('-','') #remove special character(' ', '-')

In [25]:
df.head(3)

,epoch,train/box_loss,train/seg_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP5095(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),metrics/mAP5095(M),val/box_loss,val/seg_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,1.65780,3.2735,3.2548,1.51930,0.82211,0.72367,0.69617,0.58483,0.81807,0.72011,0.69388,0.47341,0.69994,1.5850,2.6985,0.89123,0.000283,0.000283,0.000283
1,2,0.92100,1.4139,1.6942,0.98262,0.96448,0.77298,0.81965,0.69444,0.96448,0.77298,0.81941,0.66227,0.75255,1.1885,2.1867,0.93700,0.000583,0.000583,0.000583
2,3,0.83914,1.1888,1.4551,0.95567,0.86124,0.81495,0.80747,0.68315,0.87209,0.80070,0.79869,0.47669,0.71941,1.7702,2.2854,0.89041,0.000883,0.000883,0.000883


In [24]:
# df_map = df[['metrics/mAP50(B)','metrics/mAP5095(B)','metrics/mAP50(M)','metrics/mAP5095(M)']]
# df_map.head(3)

,metrics/mAP50(B),metrics/mAP5095(B),metrics/mAP50(M),metrics/mAP5095(M)
0,0.69617,0.58483,0.69388,0.47341
1,0.81965,0.69444,0.81941,0.66227
2,0.80747,0.68315,0.79869,0.47669


In [ ]:
# origin: ultralytics/ultralytics/utils/metrics.py
# def fitness(x): 
#      # Model fitness as a weighted combination of metrics 
#      w = [0.0, 0.0, 0.1, 0.9]  # weights for [P, R, mAP@0.5, mAP@0.5:0.95] 
#      return (x[:, :4] * w).sum(1) 

In [26]:
def fitness(x): 
     # Model fitness as a weighted combination of metrics 
     w = [0.1, 0.9]  # weights for [mAP@0.5(B or M), mAP@0.5:0.95(B or M)] 
     return (x[:, :2] * w).sum(1)

In [27]:
#input mAP50(B), 50-95(B), mAP50(M), 50-95(M) and add column
selected_column_B = ['metrics/mAP50(B)','metrics/mAP5095(B)']
selected_column_M = ['metrics/mAP50(M)','metrics/mAP5095(M)']

#B;Box M;Mask
df['fitness_B'] = fitness(df[selected_column_B].values)
df['fitness_M'] = fitness(df[selected_column_M].values)

In [29]:
df.head(3) #check fitness_B, fitness_M columns

,epoch,train/box_loss,train/seg_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP5095(B),metrics/precision(M),...,metrics/mAP5095(M),val/box_loss,val/seg_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2,fitness_B,fitness_M
0,1,1.65780,3.2735,3.2548,1.51930,0.82211,0.72367,0.69617,0.58483,0.81807,...,0.47341,0.69994,1.5850,2.6985,0.89123,0.000283,0.000283,0.000283,0.595964,0.495457
1,2,0.92100,1.4139,1.6942,0.98262,0.96448,0.77298,0.81965,0.69444,0.96448,...,0.66227,0.75255,1.1885,2.1867,0.93700,0.000583,0.000583,0.000583,0.706961,0.677984
2,3,0.83914,1.1888,1.4551,0.95567,0.86124,0.81495,0.80747,0.68315,0.87209,...,0.47669,0.71941,1.7702,2.2854,0.89041,0.000883,0.000883,0.000883,0.695582,0.508890


In [34]:
df.shape

(291, 22)

In [32]:
max_index_B = df['fitness_B'].idxmax() #max fitness value
max_index_M = df['fitness_M'].idxmax()

fitness_epoch_B = df.loc[max_index_B, 'epoch']
fitness_epoch_M = df.loc[max_index_M, 'epoch']

print(f"max_index_B: {max_index_B}\nfitness_epoch_B: {fitness_epoch_B}, max_fitness_B: {df['fitness_B'].max()}")
print(f"max_index_M: {max_index_M}\nfitness_epoch_M: {fitness_epoch_M}, max_fitness_M: {df['fitness_M'].max()}")


max_index_B: 286
fitness_epoch_B: 287, max_fitness_B: 0.9637340000000001
max_index_M: 245
fitness_epoch_M: 246, max_fitness_M: 0.929741
